# CWL in Polus-Plugins and Polus-Pipelines

### Definitions
* CWL: standards to *describe* tools and workflows to make them easily **portable** and **scalable**
* CommandLineTool(CTL): a *job*. In our case, a description of a Plugin
* Workflow: a group of CommandLineTools connected

### Two Objectives:
1. Create a single CommandLineTool (single job) for each plugin to be run using CWL (`polus-plugins`)
2. Once there is a CommandLineTool for each plugin in a workflow, create a CWL Workflow to run them all (`polus-pipelines`)

## Limitations

## 1. Output paths can *only* be relative paths

`outDir` can only be a relative path

Relative to: current working directory when running script

In [2]:
! pwd

/Users/camilovelezr/polus-plugins/examples_cwl


Only directory name matters

In [4]:
# p.outDir = "/Users/camilovelezr/cwl_io/output"
! ls ./output

S1_R1_C1-C11_A1_y005_x008_c000.ome.zarr S1_R1_C1-C11_A1_y005_x008_c003.ome.zarr
S1_R1_C1-C11_A1_y005_x008_c001.ome.zarr S1_R1_C1-C11_A1_y005_x008_c004.ome.zarr
S1_R1_C1-C11_A1_y005_x008_c002.ome.zarr


### Solution
* Make it explicit and *very clear* that when setting up the output directory of CLT, the `outDir` path **has to** be a relative path.

* *Even if outDir in plugin object is absolute, e.g `plugin.outDir = "/home/camilovelezr/axle/zarroutput"` only `zarroutput` will matter*

* Warning users if absolute path is passed

## 2. `Entrypoint` cannot be used

From Basic Flatfield Correction Plugin Dockerfile:
```
...
# Default command. Additional arguments are provided through the command line
ENTRYPOINT ["python3", "main.py"]
```

CWL defines a new working directory with a random name for each container it runs. Then it tries to run `main.py` in that working directory. However, `main.py` is not there since this working directory is new and only has been filled by CWL.

```
29-Nov-22 12:28:48 - cwltool  - INFO     - [job bfc.cwl] ... docker \
    run \
    -i \
    --mount=type=bind,source=/private/var/folders/fq/3xjy8gl17c7d1_j25xvpltz80000gn/T/3qvk72rb,target=/EnKELa \
    --workdir=/EnKELa \
```

```
/usr/bin/python3: can't open file '/EnKELa/main.py': [Errno 2] No such file or directory
```

**Entrypoint cannot be overridden**

From OME Zarr Converter Compute Dockerfile
```
...
CMD ["python3", "/opt/executables/main.py"] -> BaseCommand instead of Entrypoint
```

BaseCommand can be specified to CWL - *can be overridden* - and it's part of the new WIPP Plugin Schema! In this case, it also uses an **absolute** path instead of a relative path for the script

### Solution
1. Use `CMD` instead of `ENTRYPOINT` and use absolute paths. Include `CMD` in `plugin.json` as Base Command. Tested, it works.
2. Keep using `ENTRYPOINT` but only with absolute paths. Not part of current schema. Not tested, may work